In [2]:
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

import json
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)

In [408]:
I = json.dumps({'status': 'idle', 'origin': None, 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 1, 'name': 'driver_1', 'p2': None, 'p1': None, 'ctime': '2016-09-30 23:36:03.753770', 'altdest1id': None, 'destination': None, 'destinationid': None, 'location': [40.7778, -73.9448]})
I = json.loads(I)
ctime = I['ctime']
driver = I ['id']
status = I ['status']
location = I['location']
name = I ["name"]

try:
    tmp = datetime.strptime("{}".format(ctime), '%Y-%m-%d %H:%M:%S.%f')
    ctime = tmp.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
except:
    pass


In [409]:
geo_query = { "from" : 0, "size" : 1,
                      "query": {"filtered": { "filter": { "geo_distance": \
                          { "location": location,  "distance_type": "plane", "distance": "100km" }}, "query": { "query_string": { "query": "wait" }}}}}

res = es.search(index='passenger', doc_type='rolling', body=geo_query, ignore=[404, 400])

if len(res['hits']['hits'])>0: 
    passenger = res['hits']['hits'][0]["_source"]
    doc = json.dumps({"status": "pickup", "driver": driver})
    q = '{{"doc": {}}}'.format(doc)
    res = es.update(index='passenger', doc_type='rolling', id=passenger['id'], body=q, ignore=[400,404])
    doc = {"status": "pickup", "ctime": ctime, \
           "location": location, 'name': name, \
           'destination':  passenger['location'], \
           'destinationid': passenger['id']}
    if not p1:
        doc['p1'] = passenger['id']
    elif not p2:
        doc['p2'] = passenger['id']
    else:
        pass # It's full!
    doc = json.dumps(doc)
    q = '{{"doc": {}, "doc_as_upsert": "true"}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                    body=q)
    doc = {"status": "idle", "ctime": ctime, "location": location, 'name': name}
    doc = json.dumps(doc)
    q = '{{"doc": {}, "doc_as_upsert": "true"}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                    body=q)
    print doc
else:
    doc = {"ctime": ctime, "location": location}
    doc = json.dumps(doc)
    q = '{{"doc": {}}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                        body=q)
    print(doc)

{"ctime": "2016-09-30T23:36:03.753770Z", "location": [40.7778, -73.9448]}


In [37]:
last_uid = 1
q = es.get(index='passenger', doc_type='rolling', id=last_uid, ignore=[404, 400])


In [59]:
q = { 'size': 1,
    'query' : {
            'term' : { 'status' : 'arrived' }
    },
    "sort": [
    {
      "ctime": {
        "order": "asc"
      }}]
    }
res = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])


if len(res['hits']['hits']) > 0:
    res = res['hits']['hits'][0]['_source']
    msg = ["Driver {} just arrived to {}".format(\
            res['id'], res['destinationid']), res['destination']]

In [34]:
geo_query = { "from" : 0, "size" : 1,
                      "query": {
                   "filtered": {
                     "query" : {
                      "term" : {"status": "wait"}},
                     "filter": {
                        "geo_distance": {
                            "distance": '5km',
                            "distance_type": "plane", 
                            "location": [444.8076, -87.7827] }}
                }}}

res = es.search(index='passenger', doc_type='rolling', body=geo_query, ignore=[404, 400])


In [35]:
True        if res['hits'] else False

KeyError: 'hits'

In [36]:
if len(res['hits']['hits']) > 0:
        res = res['hits']['hits'][0]['_source']
        msg = ["Passenger {} just arrived to {}".format(\
            [res['id'], res['match']], res['destinationid']), res['location']]

KeyError: 'hits'

In [37]:
res

{u'error': {u'failed_shards': [{u'index': u'passenger',
    u'node': u'do-e9D6USH6zmxxtBFAYiw',
    u'reason': {u'col': 130,
     u'index': u'passenger',
     u'line': 1,
     u'reason': u'illegal longitude value [444.8076] for [geo_distance]',
     u'type': u'query_parsing_exception'},
    u'shard': 0}],
  u'grouped': True,
  u'phase': u'query',
  u'reason': u'all shards failed',
  u'root_cause': [{u'col': 130,
    u'index': u'passenger',
    u'line': 1,
    u'reason': u'illegal longitude value [444.8076] for [geo_distance]',
    u'type': u'query_parsing_exception'}],
  u'type': u'search_phase_execution_exception'},
 u'status': 400}

In [5]:
res

False